In [9]:
import os
import numpy as np
import imageio.v3 as imageio
import matplotlib.pyplot as plt
import cv2
import glob
import pandas as pd
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from matplotlib import gridspec
from scipy.signal import correlate, correlation_lags
from scipy.stats import pearsonr
from pathlib import Path
from tqdm import tqdm
from itertools import groupby
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}

plt.rcParams.update(params)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Mass analysis module!
Here all generated csv sheet will be read, and assembled in to two giant pandas dataframes: videos and hyphae.

There are two datasheets that will be read:

    - Datasheets that pertain to the properties of the videos
    - Datasheets that contain the averaged results of the data
Ideally, the first datasheet will be generated by thev VideoInfo.txt files that Morrison outputs, but there are still a lot of videos that were made before Morrison was fully operational. Later on, this document will also be able to read the TIFFs of individual hyphae to create more data.

The first question to ask: Where is the analysis data stored? 

In [2]:
directory_target = "/gpfs/scratch1/shared/amftrackflow/Rachael_set/"

## Initial reading
This is where the video properties will be read from the excel files. It is called an initial reading, as this is just the raw data, which will be processed into a more legible datasheet in the next block. 

In [ ]:
target_excels = sorted(glob.glob(directory_target + "**/*.x*"))
ini_read = pd.read_excel(target_excels[0], nrows=0)

for file in target_excels:
    read = pd.read_excel(file)
    ini_read = pd.concat([ini_read, read], ignore_index=True)

ini_read = ini_read.drop(ini_read[ini_read['Magnification'].isnull()].index)
ini_read

## Standardization
In the initial reading, the data can be from either an excel sheet, or a csv sheet. The next block will be about shaping that data into a more legible form, and doing some calculations where extra information is necessary. Right now, only excel sheets are supported, but csv support should come soon.

This is also where a whole bunch of errors can come from if data documentation was not done properly.

In [ ]:
is_excel = True

videos_data = ini_read.copy()
videos_data["Plate_nr"] = [int(plate.split('_')[-2][5:]) for plate in videos_data["Unnamed: 0"]]
videos_data["Magnification"] = [int(mag) for mag in videos_data["Magnification"]]
videos_data[["FrameRate", "FPS"][is_excel]] = [int(mag) for mag in videos_data[["FrameRate","FPS"][is_excel]]]
videos_data["Time after crossing"] = [int(mag.split(' ')[-2]) for mag in videos_data["Time after crossing"]]
videos_data["Address"] = [glob.glob(directory_target + '/' + name.split("_")[-3] + '*/' + name.split("_")[-1]) for name in videos_data["Unnamed: 0"]]
# print(videos_data["Address"].iloc[0])

videos_data = videos_data[videos_data['Address'].map(len) > 0]
videos_data["Address"] = [entry[0] for entry in videos_data["Address"]]
videos_data = videos_data.rename(columns={
    "Unnamed: 0" : "video_title",
    'Time after crossing' : 'days_old',
    'Growing temperature' : 'grow_temp',
    'Position mm' : 'xpos',
    'Unnamed: 6' : 'ypos',
    'Bright-field (BF)\nor\nFluorescence (F)' : 'mode',
    'Magnification' : 'mag',
    'FPS' : 'fps',
    'Binned (Y/N)' : 'binning',
    'Video Length (s)' : 'vid_len'
}, errors='Raise')

# Below line takes all empty binning values, and assumes no binning took place. 
# Mostly for the first few days of Rachael's dataset.
videos_data['binning'] = [np.where(entry == entry, np.where(entry == 'Y', 2, 1), 1) for entry in videos_data['binning']]
videos_data["space_res"] = 2.0 * 1.725 / videos_data['mag'] * videos_data['binning']

videos_data

## Edges datasheet creation
Each video has a number of edges, the averaged data of which is stored in the edges_data.csv file. This block of code reads that file, and creates a new row for each edge. These rows are expansions of the rows in the videos_data DataFrame from above. 

After many rounds of analysis (read: debugging the segmentation algorithm), there will be many more folders with edge data than edges in the video. By reading the edges_data.csv, only the most recently segmented edges are read. There should at some point be a purge of superfluous edge files.

In [ ]:
edges_data = videos_data.copy()
edges_data["edge_addr"] = [glob.glob(entry + '/Analysis/edge *') for entry in edges_data["Address"]]
edges_data = edges_data.explode("edge_addr")
edges_data = edges_data.drop(edges_data[edges_data['edge_addr'].isnull()].index)
edges_data.index = range(len(edges_data))
edges_data["edge_name"] = [row.split(os.sep)[-1][5:] for row in edges_data["edge_addr"]]

# print(edges_data.iloc[0])
# edges_data = edges_data[edges_data["days_old"] == 10]
edge_results = pd.DataFrame()

for index, row in edges_data.iterrows():
    edge_csv_list = row['Address'] + '/Analysis/edges_data.csv'
    if os.path.exists(edge_csv_list):
        video_edge_data = pd.read_csv(edge_csv_list)
        single_edge_data = video_edge_data[video_edge_data['edge_name'] == row['edge_name']]
        if len(single_edge_data) > 0:
            edge_results = pd.concat([edge_results, single_edge_data.set_index(pd.Index([index]))])
        row = pd.concat([row, single_edge_data])
    else:
        print(edge_csv_list)
        continue
print(edge_csv_list)

print(edges_data["Plate_nr"].unique())
edges_data = edges_data.join(edge_results, lsuffix='_l', rsuffix='_r')
edges_data = edges_data[~np.isnan(edges_data['edge_xpos_1'])]

print(edges_data.columns)

edges_counts = edges_data.pivot_table(columns=['video_title'], aggfunc='size')
videos_data['nr_of_edges'] = np.nan
for index, row in videos_data.iterrows():
    if row['video_title'] not in edges_counts.index:
        print(f"Oh no! {row['video_title']}")
        continue
    edge_count = edges_counts[row['video_title']]    
#     print(edge_count)
    videos_data.loc[index, 'nr_of_edges'] = int(edge_count)
    videos_data['coords'] = [str(i)+ str(j) for i,j in videos_data[['xpos', 'ypos']].values]

edges_data

### Width distribution

In [ ]:
print(edges_data.columns)

edges_filtered = edges_data.copy()
edges_filtered = edges_filtered[edges_filtered['mag'].ge(6)]
edges_fluo = edges_filtered[edges_filtered['mode'] == "F"]
edges_bright = edges_filtered[edges_filtered['mode'] == "BF"]

fig, ax = plt.subplots(facecolor='black')
ax.hist(edges_fluo['edge_width'], bins=30, range=(0, 20), label="Fluorescence", alpha=0.5)
ax.hist(edges_bright['edge_width'], bins=30, range=(0, 20), label= "Brightfield", alpha=0.5)
ax.set_title("Hypha widths histogram (50x mag)", c='w')
ax.set_xlabel("width $(\mu m)$", c='w')
ax.legend()



### Add number of edges in each video to videos dataframe

In [ ]:
width_pairs = []

videos_singlets = videos_data[videos_data['nr_of_edges'] == 1]
for nr in videos_singlets['Plate_nr'].unique():
    vid_single_plate = videos_singlets[videos_singlets['Plate_nr'] == nr]
    for coord in vid_single_plate['coords'].unique():
        single_edge = vid_single_plate[vid_single_plate['coords'] == coord]
        if len(single_edge) > 1:
            if single_edge['mode'].nunique() == 2:
                single_edge_bf = single_edge[single_edge['mode'] == 'BF']
                single_edge_fl = single_edge[single_edge['mode'] == 'F']
                for title_bf in single_edge_bf['video_title']:
                    edge_bf_data = edges_data[edges_data['video_title'] == title_bf]
                    edge_bf_width = edge_bf_data['edge_width'].iloc[0]
                    for title_f in single_edge_fl['video_title']:
                        edge_fl_data = edges_data[edges_data['video_title'] == title_f]
                        edge_fl_width = edge_fl_data['edge_width'].iloc[0]
                        if edge_fl_data['binning'].iloc[0] == 1:
                            edge_fl_width *= 2
                        width_pairs.append([edge_bf_width, edge_fl_width])

width_pairs = np.array(width_pairs)

fig, ax = plt.subplots(facecolor='black')
for pair in width_pairs:
    ax.scatter(pair[0], pair[1], c='tab:blue')
ax.set_ylim([3, 15])
ax.set_xlim([5, 15])
ax.set_xlabel("Bright-field edge width $(\mu m)$", c='w')
ax.set_ylabel("Fluorescence edge width $(\mu m)$", c='w')
ax.plot(np.arange(0, 20, 1), np.arange(0, 20, 1), c='black', linestyle='--', label='1:1')
ax.set_title("Comparison of edge widths", c='w')
ax.legend()


In [ ]:
cov_thresh = 0.3

edges_filtered = edges_data[edges_data['coverage_tot'].ge(cov_thresh)]

fig, ax = plt.subplots(facecolor='black')
edges_fluo = edges_filtered
print(edges_filtered["Plate_nr"].unique())
plate_nr = 558
edges_filtered_nr = edges_fluo
print(len(edges_fluo))

ax.scatter(edges_filtered_nr['ypos'], edges_filtered_nr['speed_mean'], s=10, alpha=0.1, label='mean')
# ax.scatter(edges_filtered_nr['ypos'], edges_filtered_nr['speed_left'], s=10, alpha=0.1, label='to root')
ax.axhline(c='black', linestyle='--')

# ax.scatter(edges_filtered_nr['edge_width'], edges_filtered_nr['speed_mean'], s=10, alpha=0.2, label='mean')
y_series = sorted(edges_filtered_nr['ypos'].unique())
y_r_mean = np.array([edges_filtered_nr['speed_mean'][edges_filtered['ypos'] == y].mean() for y in y_series])
# y_r_std = np.array([edges_filtered_nr['speed_right'][edges_filtered['ypos'] == y].std() for y in y_series])
# y_l_mean = np.array([edges_filtered_nr['speed_left'][edges_filtered['ypos'] == y].mean() for y in y_series])
# y_l_std = np.array([edges_filtered_nr['speed_left'][edges_filtered['ypos'] == y].std() for y in y_series])

y_r = pd.Series(data=y_r_mean, index = y_series)
# y_l = pd.Series(data=y_l_mean, index = y_series)

ax.plot(y_r.rolling(20).mean(), c='black', label='Rolling right average')
# ax.fill_between(y_series, 
#                       y_r_mean + y_r_std, 
#                       y_r_mean - y_r_std, 
#                       alpha=0.5, facecolor='tab:blue')
# ax.plot(y_l.rolling(20).mean(), c='tab:orange', label='Rolling left average')
# ax.fill_between(y_series, 
#                       y_l_mean + y_l_std, 
#                       y_l_mean - y_l_std, 
#                       alpha=0.5, facecolor='tab:orange')

# ax.set_xlim((0, 20))
ax.set_ylabel("Speed $(\mu m /s )$", c='w')
ax.set_xlabel("y-position (mm) (tip -> root)", c='w')
ax.set_title(f"Scatter of y-position and speeds", c='w')
ax.set_xlim((15000, 50000))
ax.legend()

### Positional distribution

In [ ]:


for plate_nr in videos_data['Plate_nr'].unique():
# plate_nr = 452

    videos_filt = videos_data[videos_data['Plate_nr'] == plate_nr]
    edges_filt = edges_data[edges_data['Plate_nr'] == plate_nr]
    edges_filt = edges_filt[edges_filt['mode'] == "F"]
#     edges_filt = edges_filt[edges_filt['mag'] == 50]
    arr_lengths_r = np.arctan(edges_filt['speed_right'] / 10) * 6.5
    arr_lengths_l = np.arctan(edges_filt['speed_left'] / 10) * 6.5

    edge_ori_x = edges_filt['edge_xpos_2'] - edges_filt['edge_xpos_1']
    edge_ori_y = edges_filt['edge_ypos_2'] - edges_filt['edge_ypos_1']
    edge_ori_theta = -np.arctan2(edge_ori_x, edge_ori_y)
    # print(np.array(edge_ori_theta))

    xpos_4 = videos_filt["xpos"][videos_filt["mag"] == 4]
    ypos_4 = -videos_filt["ypos"][videos_filt["mag"] == 4]

    xpos_50 = videos_filt["xpos"][videos_filt["mag"] == 50]
    ypos_50 = -videos_filt["ypos"][videos_filt["mag"] == 50]

    # theta = np.linspace(0, 2*np.pi, 41, endpoint=True)
    # radii, bin_edges = np.histogram(np.array(edge_ori_theta), bins=theta)
    # width = (2*np.pi) / 41

    # fig2 = plt.figure(facecolor='black')
    # ax2 = plt.subplot(111, polar=True)
    # bars = ax2.bar(theta[:-1], radii, width=width, bottom=50)
    # ax2.set_title("Orientation histogram", c='w')

    fig, ax = plt.subplots(figsize=(16, 9), facecolor='black')
    ax.scatter(xpos_50, ypos_50, c='tab:orange', s=2*12, label='50x mag')
    ax.scatter(xpos_4, ypos_4, c='tab:green', s=2*12, label='4x mag', alpha=0.5)

    ax.quiver(edges_filt['xpos'], -edges_filt['ypos'], 
              arr_lengths_r*np.cos(edge_ori_theta), 
              arr_lengths_r*np.sin(edge_ori_theta),
              scale=300, width=0.0015, alpha=1.0, color='tab:blue')
    ax.quiver(edges_filt['xpos'], -edges_filt['ypos'], 
              arr_lengths_l*np.cos(edge_ori_theta), 
              arr_lengths_l*np.sin(edge_ori_theta),
              scale=300, width=0.0015, alpha=1.0, color='tab:orange')

    ax.legend()
    ax.set_title(f"Fluorescence videos bi-directional velocity of {plate_nr}")
    ax.set_xlim((-5000, 60000))
    ax.set_ylim([-50000, -15000])
    ax.set_aspect('equal')
    ax.set_ylabel("ypos", c='w')
    ax.set_xlabel("xpos", c='w')
    fig.tight_layout()
    fig.savefig(f"/gpfs/home6/svstaalduine/plot_outs/plate_{plate_nr}_vidposs.png", transparent=True)

In [ ]:
import matplotlib.patches as mpatches

forw=True
back=True
mean=True

labels = []
def set_axis_style(ax, labels):
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Sample name')
    
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

# [452 462 510 537 530 545 532 527 528 552 558] are the plate numbers
plate_interest= None

edges_cov = edges_data[edges_data['coverage_tot'].ge(cov_thresh)]
edges_cov = edges_cov[edges_cov['mode'] == 'BF']
if plate_interest is not None:
    edges_cov = edges_cov[edges_cov['Plate_nr'] == plate_interest]

# width_bins = [0, 4, 5,6,7,8,9,10, 11, 12, 13]
width_bins = np.linspace(15000, 50000, 12)
edges_cov['Binned'] = pd.cut(edges_cov['ypos'], width_bins)
edges_binned = edges_cov.sort_values(by=['Binned'])
plate_ages = edges_binned['Binned'].unique()
bin_lens = []
print(plate_ages)

fig, ax = plt.subplots(facecolor='black', figsize=(10, 6))
if forw:
    spd_list_forw = []
    for age in plate_ages:
        edges_filt = edges_cov[edges_cov['Binned'] == age]
        edges_filt = edges_filt.fillna(0)
        spd_list_forw.append(np.array(edges_filt['flux_max']))
    parts = ax.violinplot(spd_list_forw, showmeans=False, showextrema=False)
    for pc in parts['bodies']:
        pc.set_facecolor('tab:orange')
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    add_label(parts, 'To tip')
    
if back:
    spd_list_back = []
    for age in plate_ages:
        edges_filt = edges_cov[edges_cov['Binned'] == age]
        edges_filt = edges_filt.fillna(0)
        spd_list_back.append(np.array(edges_filt['flux_min']))
    parts = ax.violinplot(spd_list_back, showmeans=False, showextrema=False)
    for pc in parts['bodies']:
        pc.set_facecolor('tab:blue')
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    add_label(parts, 'To root')

if mean:
    spd_list_mean = []
    for age in plate_ages:
        edges_filt = edges_cov[edges_cov['Binned'] == age]
        edges_filt = edges_filt.fillna(0)
        spd_list_mean.append(np.array(edges_filt['flux_avg']))
        bin_lens.append(len(edges_filt))
    print(bin_lens)
    parts = ax.violinplot(spd_list_mean, showmeans=False, showextrema=False)
    for pc in parts['bodies']:
        pc.set_facecolor('tab:red')
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    add_label(parts, 'Mean')

ax.axhline(c='black', linestyle='--')
    
means = np.array([(i, np.mean(spds)) for i, spds in enumerate(spd_list_mean)]).T

mean_ax = ax.scatter(means[0]+1, means[1], c='black', s=10, label='mean')
ax.legend(*zip(*labels))
plate_ages = [f"{width.right / 1000 : 0.3}" for width in plate_ages]


set_axis_style(ax, plate_ages)
ax.set_ylabel("Flux $(\mu m / s)$", c='w')
# ax.set_xlabel("width $(\mu m)$", c='w')
ax.set_xlabel("y-position $(mm)$, tip --> root", c='w')
# ax.set_ylim([-35, 35])
ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(np.arange(len(bin_lens)) + 1)
ax2.set_xticklabels(bin_lens)
ax2.set_xlabel("Number of edges", c='w')
ax.set_title("Flow velocities average flux (fluorescence videos only)", c='w')
fig.tight_layout()


## 50x and 4x comparisons

First we filter out many edges with no coverage

In [7]:

videos_filt = videos_data[videos_data['mag'] == 4]
videos_filt.index = range(len(videos_filt))
# edges_filt = edges_filt[edges_filt['coverage_tot'] > 0.5]
# edges_filt = edges_filt[edges_filt['coverage_left'] > 0.7]
# edges_filt = edges_filt[edges_filt['speed_right_std'] < 0.5]
# edges_filt['speed_range'] = edges_filt['speed_right'] - edges_filt['speed_left']
# edges_filt = edges_filt[edges_filt['speed_range'] > 6]

print(videos_filt.columns)
# print(edges_filt['edge_name_r'])
videos_filt

Index(['video_title', 'Treatment', 'Strain', 'days_old', 'grow_temp', 'xpos',
       'ypos', 'dcenter mm', 'droot mm', 'mode', 'mag', 'fps', 'Comments',
       'binning', 'vid_len', 'Plate_nr', 'Address', 'space_res', 'nr_of_edges',
       'coords'],
      dtype='object')


,video_title,Treatment,Strain,days_old,grow_temp,xpos,ypos,dcenter mm,droot mm,mode,mag,fps,Comments,binning,vid_len,Plate_nr,Address,space_res,nr_of_edges,coords
0,20221026_plate452_16,1%P100&N,C2,10,25.0,11060,46280,-33.0,-3.0,F,4,10,SET 1\n30 seconds. Part of network out of focus,1,NaN,452,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,0.8625,3.0,1106046280
1,20221026_plate452_24,1%P100&N,C2,10,25.0,13960,44040,-30.0,-1.0,F,4,10,SET 2\n30 seconds,1,NaN,452,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,0.8625,3.0,1396044040
2,20221026_plate452_35,1%P100&N,C2,10,25.0,7460,39680,-37.0,4.0,F,4,10,SET 3\n60 seconds. Can see flow movement,1,NaN,452,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,0.8625,1.0,746039680
3,20221026_plate452_43,1%P100&N,C2,10,25.0,11260,42720,-33.0,1.0,F,4,10,"SET 4\nGood FOV, can see different hyphae in f...",1,NaN,452,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,0.8625,2.0,1126042720
4,20221026_plate452_55,1%P100&N,C2,10,25.0,7520,42860,-36.0,0.0,F,4,10,SET 6\n60 seconds,1,NaN,452,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,0.8625,2.0,752042860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20230201_Plate558_13,1%P100%N,C2,23,25.0,55740,44420,12.0,-1.0,F,4,10,NaN,2,60.0,558,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,1.725,3.0,5574044420
99,20230201_Plate558_22,1%P100%N,C2,23,25.0,61140,35260,17.0,8.0,F,4,10,NaN,2,60.0,558,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,1.725,2.0,6114035260
100,20230201_Plate558_27,1%P100%N,C2,23,25.0,39140,43760,-5.0,0.0,F,4,10,NaN,2,60.0,558,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,1.725,3.0,3914043760
101,20230201_Plate558_32,1%P100%N,C2,23,25.0,36980,28300,-7.0,15.0,F,4,10,NaN,2,60.0,558,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,1.725,3.0,3698028300


In [8]:
video_4x_index = 43

loc_tolerance = 1000
for video_4x_index in tqdm(range(103)):
    video_data = videos_filt.iloc[video_4x_index]
    img_4x = imageio.imread(glob.glob(video_data['Address'] + "/Img/Ba*.tif*")[0])

    x_adj = 100-750+400+75
    y_adj = -500

#     print(img_4x.shape)

    edges_4x = edges_data[edges_data['Plate_nr'] == video_data['Plate_nr']]
    edges_4x = edges_4x[edges_4x['mode'] == 'F']
    edges_4x = edges_4x[edges_4x['xpos'].between(video_data['xpos']- loc_tolerance, video_data['xpos']+ loc_tolerance)]
    edges_4x = edges_4x[edges_4x['ypos'].between(video_data['ypos']- loc_tolerance, video_data['ypos']+ loc_tolerance)]

    fig, ax = plt.subplots( figsize = (16, 9), facecolor='black')
    # ax.imshow(img_4x, extent=((video_data['xpos']- img_4x.shape[1]*0.5) + x_adj, (video_data['xpos'] + img_4x.shape[1]*0.5)+x_adj, 
    #                           (-video_data['ypos']) + y_adj, (-1*(video_data['ypos'] - img_4x.shape[0]))+y_adj))
    ax.scatter(edges_4x['xpos'], -edges_4x['ypos'], label='50x video')
    ax.scatter(video_data['xpos'], -video_data['ypos'], label='4x video')

    for index, row in edges_4x.iterrows():
        if row['mag'] == 4:
            continue
        arrow_start =  np.array([(row['xpos'] + row['space_res'] *row['edge_ypos_2'], 
                        (row['ypos'] + row['space_res'] *row['edge_xpos_2'])*-1)])[0]
        arrow_end =    np.array([(row['xpos'] + row['space_res'] *row['edge_ypos_1'], 
                        (row['ypos'] + row['space_res'] *row['edge_xpos_1'])*-1)])[0] - arrow_start
        ax.quiver(arrow_start[0], arrow_start[1], arrow_end[0], arrow_end[1], angles='xy', color=['tab:green', 'gray'][row['mag'] == 50], scale_units='xy', scale=1)
    #     print(arrow_start, arrow_end)
    ax.set_xlim((video_data['xpos']- img_4x.shape[1]*0.5) + x_adj, (video_data['xpos'] + img_4x.shape[1]*0.5)+x_adj)
    ax.set_ylim((-video_data['ypos']) + y_adj, (-1*(video_data['ypos'] - img_4x.shape[0]))+y_adj)

    used_videos = [row.split('_')[-1] for row in edges_4x['video_title'].unique()]
#     print(used_videos)
    ax.set_title(f"4x overview of {video_data['video_title']}, \n with 50x videos {used_videos[1:]}", c='w')
    ax.set_aspect('equal')
    ax.legend()
    fig.tight_layout()
    fig.savefig(f"/gpfs/home6/svstaalduine/plot_outs/overviews_4x/{video_data['video_title']}_4x_overview.png", transparent=True)
    plt.close('all')
# edges_4x

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [01:37<00:00,  1.06it/s]
